In [1]:
!which python

/sw/centos/anaconda3/2019.10/bin/python


In [33]:
!pip install xgboost
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/centos/anaconda3/2019.10/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/centos/anaconda3/2019.10/bin/python3 -m pip install --upgrade pip' command.


In [34]:
%config IPCompleter.greedy=True
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib as matplot
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")

# from keras import Sequential
# from keras.models import Model, load_model
# from keras.layers import *
# from keras.callbacks import ModelCheckpoint
# from keras import regularizers

from sklearn.metrics import *
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,normalize
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model


import xgboost, lightgbm
# from mlxtend.classifier import EnsembleVoteClassifier


# Preprocessing

In [35]:
train = pd.read_csv('UNSW-NB15_4_train.csv')
test = pd.read_csv('UNSW-NB15_4_test.csv')
combined_data = pd.concat([train, test])

In [36]:
combined_data

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label,src_ip_with_port,trg_ip_with_port
0,149.171.126.10,47439,175.45.176.0,53,udp,INT,0.000007,264,0,60,...,43,37,37,37,19,43,NaN,0,149.171.126.10:47439,175.45.176.0:53
1,59.166.0.9,15662,149.171.126.4,13946,tcp,FIN,0.053957,2646,25684,31,...,6,6,2,1,1,1,NaN,0,59.166.0.9:15662,149.171.126.4:13946
2,59.166.0.8,5516,149.171.126.2,53,udp,CON,0.001023,146,178,31,...,2,1,8,1,1,2,NaN,0,59.166.0.8:5516,149.171.126.2:53
3,59.166.0.3,23879,149.171.126.6,22905,udp,CON,0.305048,520,304,31,...,9,4,5,1,1,4,NaN,0,59.166.0.3:23879,149.171.126.6:22905
4,175.45.176.0,1043,149.171.126.10,53,udp,INT,0.000009,114,0,254,...,14,14,14,14,9,14,Generic,1,175.45.176.0:1043,149.171.126.10:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88004,175.45.176.1,47439,149.171.126.14,53,udp,INT,0.000009,114,0,254,...,30,18,18,18,18,30,Generic,1,175.45.176.1:47439,149.171.126.14:53
88005,59.166.0.4,33468,149.171.126.1,31029,tcp,FIN,0.027027,2542,22128,31,...,8,3,2,1,1,1,NaN,0,59.166.0.4:33468,149.171.126.1:31029
88006,59.166.0.1,58776,149.171.126.7,22,tcp,FIN,10.215708,36536,77060,31,...,1,6,2,1,1,2,NaN,0,59.166.0.1:58776,149.171.126.7:22
88007,149.171.126.14,47439,175.45.176.1,53,udp,INT,0.000003,264,0,60,...,31,19,19,19,19,31,NaN,0,149.171.126.14:47439,175.45.176.1:53


In [37]:
from collections import Counter
label_counter = Counter(list(combined_data['Label']))
print(label_counter)

Counter({0: 351150, 1: 88894})


In [38]:
label_counter = Counter(list(train['attack_cat']))
print(label_counter)

Counter({nan: 280919, 'Generic': 49475, 'Exploits': 9178, ' Fuzzers ': 4293, 'DoS': 3946, ' Reconnaissance ': 2818, 'Backdoor': 534, 'Analysis': 533, ' Shellcode ': 299, 'Worms': 40})


In [39]:
# remove field: 'attack_cat', 'Stime', 'Ltime', 'ct_ftp_cmd'
train = train.drop(['attack_cat', 'Stime', 'Ltime', 'ct_ftp_cmd'], axis=1)
test = test.drop(['attack_cat', 'Stime', 'Ltime', 'ct_ftp_cmd'], axis=1)

In [40]:
# remove ip address field: 'srcip', 'sport', 'dstip', 'dsport'
train = train.drop(['srcip', 'sport', 'dstip', 'dsport'], axis=1)
test = test.drop(['srcip', 'sport', 'dstip', 'dsport'], axis=1)

In [41]:
train.head()

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label,src_ip_with_port,trg_ip_with_port
0,udp,INT,0.000007,264,0,60,0,0,0,dns,...,43,43,37,37,37,19,43,0,149.171.126.10:47439,175.45.176.0:53
1,tcp,FIN,0.053957,2646,25684,31,29,7,15,-,...,2,6,6,2,1,1,1,0,59.166.0.9:15662,149.171.126.4:13946
2,udp,CON,0.001023,146,178,31,29,0,0,dns,...,5,2,1,8,1,1,2,0,59.166.0.8:5516,149.171.126.2:53
3,udp,CON,0.305048,520,304,31,29,0,0,-,...,5,9,4,5,1,1,4,0,59.166.0.3:23879,149.171.126.6:22905
4,udp,INT,0.000009,114,0,254,0,0,0,dns,...,14,14,14,14,14,9,14,1,175.45.176.0:1043,149.171.126.10:53


In [42]:
# The remaining nominal field are: 'proto', 'state', 'service'
le = LabelEncoder()
train['proto'] = le.fit_transform(train['proto'])
test['proto'] = le.transform(test['proto'])
train['service'] = le.fit_transform(train['service'])
test['service'] = le.transform(test['service'])
train['state'] = le.fit_transform(train['state'])
test['state'] = le.transform(test['state'])

In [43]:
train.head()

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label,src_ip_with_port,trg_ip_with_port
0,119,5,0.000007,264,0,60,0,0,0,2,...,43,43,37,37,37,19,43,0,149.171.126.10:47439,175.45.176.0:53
1,113,4,0.053957,2646,25684,31,29,7,15,0,...,2,6,6,2,1,1,1,0,59.166.0.9:15662,149.171.126.4:13946
2,119,1,0.001023,146,178,31,29,0,0,2,...,5,2,1,8,1,1,2,0,59.166.0.8:5516,149.171.126.2:53
3,119,1,0.305048,520,304,31,29,0,0,0,...,5,9,4,5,1,1,4,0,59.166.0.3:23879,149.171.126.6:22905
4,119,5,0.000009,114,0,254,0,0,0,2,...,14,14,14,14,14,9,14,1,175.45.176.0:1043,149.171.126.10:53


In [44]:
test.head()

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label,src_ip_with_port,trg_ip_with_port
0,119,5,0.000009,114,0,254,0,0,0,2,...,12,12,12,12,12,12,12,1,175.45.176.0:47439,149.171.126.10:53
1,119,1,0.001063,130,162,31,29,0,0,2,...,1,3,2,4,1,1,2,0,59.166.0.5:64199,149.171.126.1:53
2,113,4,0.004815,1470,1728,31,29,5,4,0,...,7,2,1,1,1,1,1,0,59.166.0.6:62877,149.171.126.1:5190
3,119,5,0.000009,264,0,60,0,0,0,2,...,23,23,19,19,19,19,23,0,149.171.126.10:47439,175.45.176.0:53
4,119,1,0.004537,568,320,31,29,0,0,0,...,3,5,4,2,1,1,3,0,59.166.0.7:43618,149.171.126.8:111


## Load ip2id mapping

In [45]:
import pickle
with open('vocab/dataset4_ip2id.pickle', 'rb') as handle:
    ip2id = pickle.load(handle)
with open('vocab/dataset4_id2ip.pickle', 'rb') as handle:
    id2ip = pickle.load(handle)


In [46]:
assert len(ip2id) == len(id2ip)

## Load node feature score from result of oddBall

In [16]:
# format: {node_id: out_score}
node_out_score_cliquestar = dict()
with open('./oddball/code/result_1.txt', 'r') as f:
    for line in f:
        node_id = int(line.split()[0])
        out_score = float(line.split()[1])
        node_out_score_cliquestar[node_id] = out_score
print("Load {} node's cliquestar out score".format(len(node_out_score_cliquestar)))

Load 327626 node's cliquestar out score


In [47]:
# format: {node_id: out_score}
node_out_score_heavyvicinity = dict()
with open('./oddball/code/result_2.txt', 'r') as f:
    for line in f:
        node_id = int(line.split()[0])
        out_score = float(line.split()[1])
        node_out_score_heavyvicinity[node_id] = out_score
print("Load {} node's HeavyVicinity out score".format(len(node_out_score_heavyvicinity)))

Load 327626 node's HeavyVicinity out score


In [48]:
# format: {node_id: out_score}
node_out_score_dominantPair = dict()
with open('./oddball/code/result_3.txt', 'r') as f:
    for line in f:
        node_id = int(line.split()[0])
        out_score = float(line.split()[1])
        node_out_score_dominantPair[node_id] = out_score
print("Load {} node's DominantPair out score".format(len(node_out_score_dominantPair)))

Load 327626 node's DominantPair out score


## Load node feature score

In [49]:
# add a new column with default value
train['src_score_cs'] = 0.0
test['src_score_cs'] = 0.0
train['src_score_hv'] = 0.0
test['src_score_hv'] = 0.0
train['src_score_dp'] = 0.0
test['src_score_dp'] = 0.0

In [50]:
# update the score iteratively for each row
for idx in train.index:
    current_src_node_ip = train.at[idx, 'src_ip_with_port']
    current_trg_node_ip = train.at[idx, 'trg_ip_with_port']
    train.at[idx, 'src_score_cs'] = node_out_score_cliquestar[ip2id[current_src_node_ip]]
    train.at[idx, 'src_score_hv'] = node_out_score_heavyvicinity[ip2id[current_src_node_ip]]
    train.at[idx, 'src_score_dp'] = node_out_score_dominantPair[ip2id[current_src_node_ip]]
    if (idx+1) % 100000 == 0:
        print("Processed {} lines".format(idx+1))
print("Totally {} lines.".format(idx+1))

Processed 100000 lines
Processed 200000 lines
Processed 300000 lines
Totally 352035 lines.


In [51]:
train

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label,src_ip_with_port,trg_ip_with_port,src_score_cs,src_score_hv,src_score_dp
0,119,5,0.000007,264,0,60,0,0,0,2,...,37,37,19,43,0,149.171.126.10:47439,175.45.176.0:53,3.814511e-07,148645.711848,711.946633
1,113,4,0.053957,2646,25684,31,29,7,15,0,...,2,1,1,1,0,59.166.0.9:15662,149.171.126.4:13946,3.814511e-07,0.009156,0.002316
2,119,1,0.001023,146,178,31,29,0,0,2,...,8,1,1,2,0,59.166.0.8:5516,149.171.126.2:53,3.814511e-07,0.009156,0.002316
3,119,1,0.305048,520,304,31,29,0,0,0,...,5,1,1,4,0,59.166.0.3:23879,149.171.126.6:22905,3.814511e-07,0.009156,0.002316
4,119,5,0.000009,114,0,254,0,0,0,2,...,14,14,9,14,1,175.45.176.0:1043,149.171.126.10:53,3.814511e-07,131998.841329,671.010962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352030,119,5,0.000004,264,0,60,0,0,0,2,...,19,19,19,25,0,149.171.126.14:47439,175.45.176.1:53,3.814511e-07,180806.410499,784.862459
352031,113,4,0.019771,1540,1644,31,29,4,4,0,...,8,6,1,6,0,59.166.0.5:45669,149.171.126.2:6881,4.104405e-05,0.042633,0.103402
352032,113,4,0.053166,2974,34222,31,29,7,18,0,...,3,1,1,2,0,59.166.0.5:61025,149.171.126.8:23273,3.814511e-07,0.009156,0.002316
352033,113,4,1.239816,572,768,254,252,3,4,3,...,1,1,1,1,1,175.45.176.3:28789,149.171.126.13:21,3.814511e-07,0.009156,0.002316


In [52]:
# update the score iteratively for each row
for idx in test.index:
    current_src_node_ip = test.at[idx, 'src_ip_with_port']
    current_trg_node_ip = test.at[idx, 'trg_ip_with_port']
    test.at[idx, 'src_score_cs'] = node_out_score_cliquestar[ip2id[current_src_node_ip]]
    test.at[idx, 'src_score_hv'] = node_out_score_heavyvicinity[ip2id[current_src_node_ip]]
    test.at[idx, 'src_score_dp'] = node_out_score_dominantPair[ip2id[current_src_node_ip]]
    if (idx+1) % 100000 == 0:
        print("Processed {} lines".format(idx+1))
print("Totally {} lines.".format(idx+1))

Totally 88009 lines.


In [53]:
test

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label,src_ip_with_port,trg_ip_with_port,src_score_cs,src_score_hv,src_score_dp
0,119,5,0.000009,114,0,254,0,0,0,2,...,12,12,12,12,1,175.45.176.0:47439,149.171.126.10:53,3.814511e-07,123611.255313,649.383522
1,119,1,0.001063,130,162,31,29,0,0,2,...,4,1,1,2,0,59.166.0.5:64199,149.171.126.1:53,3.814511e-07,0.009156,0.002316
2,113,4,0.004815,1470,1728,31,29,5,4,0,...,1,1,1,1,0,59.166.0.6:62877,149.171.126.1:5190,3.814511e-07,0.009156,0.002316
3,119,5,0.000009,264,0,60,0,0,0,2,...,19,19,19,23,0,149.171.126.10:47439,175.45.176.0:53,3.814511e-07,148645.711848,711.946633
4,119,1,0.004537,568,320,31,29,0,0,0,...,2,1,1,3,0,59.166.0.7:43618,149.171.126.8:111,3.814511e-07,0.009156,0.002316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88004,119,5,0.000009,114,0,254,0,0,0,2,...,18,18,18,30,1,175.45.176.1:47439,149.171.126.14:53,1.698245e-05,84999.148651,762.341680
88005,113,4,0.027027,2542,22128,31,29,7,14,0,...,2,1,1,1,0,59.166.0.4:33468,149.171.126.1:31029,3.814511e-07,0.009156,0.002316
88006,113,4,10.215708,36536,77060,31,29,77,177,11,...,2,1,1,2,0,59.166.0.1:58776,149.171.126.7:22,1.698245e-05,0.024779,0.051548
88007,119,5,0.000003,264,0,60,0,0,0,2,...,19,19,19,31,0,149.171.126.14:47439,175.45.176.1:53,3.814511e-07,180806.410499,784.862459


In [54]:
X_train_withoutgraph = train.drop(['src_ip_with_port','trg_ip_with_port', 'Label', 'src_score_cs', 'src_score_hv', 'src_score_dp'], axis=1).fillna(0)
y_train_withoutgraph = train.loc[:,['Label']]
X_test_withoutgraph = test.drop(['src_ip_with_port','trg_ip_with_port', 'Label', 'src_score_cs', 'src_score_hv', 'src_score_dp'], axis=1).fillna(0)
y_test_withoutgraph = test.loc[:,['Label']]
# CliqueStar
X_train_CS = train.drop(['src_ip_with_port','trg_ip_with_port', 'Label', 'src_score_hv', 'src_score_dp'], axis=1).fillna(0)
y_train_CS = train.loc[:,['Label']]
X_test_CS = test.drop(['src_ip_with_port','trg_ip_with_port', 'Label', 'src_score_hv', 'src_score_dp'], axis=1).fillna(0)
y_test_CS = test.loc[:,['Label']]
# HeavyVicinity
X_train_HV = train.drop(['src_ip_with_port','trg_ip_with_port', 'Label', 'src_score_cs', 'src_score_dp'], axis=1).fillna(0)
y_train_HV = train.loc[:,['Label']]
X_test_HV = test.drop(['src_ip_with_port','trg_ip_with_port', 'Label', 'src_score_cs', 'src_score_dp'], axis=1).fillna(0)
y_test_HV = test.loc[:,['Label']]
# DominantPair
X_train_DP = train.drop(['src_ip_with_port','trg_ip_with_port', 'Label', 'src_score_cs', 'src_score_hv'], axis=1).fillna(0)
y_train_DP = train.loc[:,['Label']]
X_test_DP = test.drop(['src_ip_with_port','trg_ip_with_port', 'Label', 'src_score_cs', 'src_score_hv'], axis=1).fillna(0)
y_test_DP = test.loc[:,['Label']]
# All
X_train_All = train.drop(['src_ip_with_port','trg_ip_with_port', 'Label'], axis=1).fillna(0)
y_train_All = train.loc[:,['Label']]
X_test_All = test.drop(['src_ip_with_port','trg_ip_with_port', 'Label'], axis=1).fillna(0)
y_test_All = test.loc[:,['Label']]

In [55]:
X_train_withoutgraph

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
0,119,5,0.000007,264,0,60,0,0,0,2,...,0,0.0,0.0,43,43,37,37,37,19,43
1,113,4,0.053957,2646,25684,31,29,7,15,0,...,0,0.0,0.0,2,6,6,2,1,1,1
2,119,1,0.001023,146,178,31,29,0,0,2,...,0,0.0,0.0,5,2,1,8,1,1,2
3,119,1,0.305048,520,304,31,29,0,0,0,...,0,0.0,0.0,5,9,4,5,1,1,4
4,119,5,0.000009,114,0,254,0,0,0,2,...,2,0.0,0.0,14,14,14,14,14,9,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352030,119,5,0.000004,264,0,60,0,0,0,2,...,0,0.0,0.0,25,25,19,19,19,19,25
352031,113,4,0.019771,1540,1644,31,29,4,4,0,...,0,0.0,0.0,14,19,13,8,6,1,6
352032,113,4,0.053166,2974,34222,31,29,7,18,0,...,0,0.0,0.0,9,3,3,3,1,1,2
352033,113,4,1.239816,572,768,254,252,3,4,3,...,1,0.0,0.0,1,1,1,1,1,1,1


In [56]:
X_train_CS

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,ct_flw_http_mthd,is_ftp_login,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,src_score_cs
0,119,5,0.000007,264,0,60,0,0,0,2,...,0.0,0.0,43,43,37,37,37,19,43,3.814511e-07
1,113,4,0.053957,2646,25684,31,29,7,15,0,...,0.0,0.0,2,6,6,2,1,1,1,3.814511e-07
2,119,1,0.001023,146,178,31,29,0,0,2,...,0.0,0.0,5,2,1,8,1,1,2,3.814511e-07
3,119,1,0.305048,520,304,31,29,0,0,0,...,0.0,0.0,5,9,4,5,1,1,4,3.814511e-07
4,119,5,0.000009,114,0,254,0,0,0,2,...,0.0,0.0,14,14,14,14,14,9,14,3.814511e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352030,119,5,0.000004,264,0,60,0,0,0,2,...,0.0,0.0,25,25,19,19,19,19,25,3.814511e-07
352031,113,4,0.019771,1540,1644,31,29,4,4,0,...,0.0,0.0,14,19,13,8,6,1,6,4.104405e-05
352032,113,4,0.053166,2974,34222,31,29,7,18,0,...,0.0,0.0,9,3,3,3,1,1,2,3.814511e-07
352033,113,4,1.239816,572,768,254,252,3,4,3,...,0.0,0.0,1,1,1,1,1,1,1,3.814511e-07


In [57]:
X_train_HV

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,ct_flw_http_mthd,is_ftp_login,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,src_score_hv
0,119,5,0.000007,264,0,60,0,0,0,2,...,0.0,0.0,43,43,37,37,37,19,43,148645.711848
1,113,4,0.053957,2646,25684,31,29,7,15,0,...,0.0,0.0,2,6,6,2,1,1,1,0.009156
2,119,1,0.001023,146,178,31,29,0,0,2,...,0.0,0.0,5,2,1,8,1,1,2,0.009156
3,119,1,0.305048,520,304,31,29,0,0,0,...,0.0,0.0,5,9,4,5,1,1,4,0.009156
4,119,5,0.000009,114,0,254,0,0,0,2,...,0.0,0.0,14,14,14,14,14,9,14,131998.841329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352030,119,5,0.000004,264,0,60,0,0,0,2,...,0.0,0.0,25,25,19,19,19,19,25,180806.410499
352031,113,4,0.019771,1540,1644,31,29,4,4,0,...,0.0,0.0,14,19,13,8,6,1,6,0.042633
352032,113,4,0.053166,2974,34222,31,29,7,18,0,...,0.0,0.0,9,3,3,3,1,1,2,0.009156
352033,113,4,1.239816,572,768,254,252,3,4,3,...,0.0,0.0,1,1,1,1,1,1,1,0.009156


In [58]:
X_train_DP

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,ct_flw_http_mthd,is_ftp_login,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,src_score_dp
0,119,5,0.000007,264,0,60,0,0,0,2,...,0.0,0.0,43,43,37,37,37,19,43,711.946633
1,113,4,0.053957,2646,25684,31,29,7,15,0,...,0.0,0.0,2,6,6,2,1,1,1,0.002316
2,119,1,0.001023,146,178,31,29,0,0,2,...,0.0,0.0,5,2,1,8,1,1,2,0.002316
3,119,1,0.305048,520,304,31,29,0,0,0,...,0.0,0.0,5,9,4,5,1,1,4,0.002316
4,119,5,0.000009,114,0,254,0,0,0,2,...,0.0,0.0,14,14,14,14,14,9,14,671.010962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352030,119,5,0.000004,264,0,60,0,0,0,2,...,0.0,0.0,25,25,19,19,19,19,25,784.862459
352031,113,4,0.019771,1540,1644,31,29,4,4,0,...,0.0,0.0,14,19,13,8,6,1,6,0.103402
352032,113,4,0.053166,2974,34222,31,29,7,18,0,...,0.0,0.0,9,3,3,3,1,1,2,0.002316
352033,113,4,1.239816,572,768,254,252,3,4,3,...,0.0,0.0,1,1,1,1,1,1,1,0.002316


In [59]:
X_train_All

,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,...,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,src_score_cs,src_score_hv,src_score_dp
0,119,5,0.000007,264,0,60,0,0,0,2,...,43,43,37,37,37,19,43,3.814511e-07,148645.711848,711.946633
1,113,4,0.053957,2646,25684,31,29,7,15,0,...,2,6,6,2,1,1,1,3.814511e-07,0.009156,0.002316
2,119,1,0.001023,146,178,31,29,0,0,2,...,5,2,1,8,1,1,2,3.814511e-07,0.009156,0.002316
3,119,1,0.305048,520,304,31,29,0,0,0,...,5,9,4,5,1,1,4,3.814511e-07,0.009156,0.002316
4,119,5,0.000009,114,0,254,0,0,0,2,...,14,14,14,14,14,9,14,3.814511e-07,131998.841329,671.010962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352030,119,5,0.000004,264,0,60,0,0,0,2,...,25,25,19,19,19,19,25,3.814511e-07,180806.410499,784.862459
352031,113,4,0.019771,1540,1644,31,29,4,4,0,...,14,19,13,8,6,1,6,4.104405e-05,0.042633,0.103402
352032,113,4,0.053166,2974,34222,31,29,7,18,0,...,9,3,3,3,1,1,2,3.814511e-07,0.009156,0.002316
352033,113,4,1.239816,572,768,254,252,3,4,3,...,1,1,1,1,1,1,1,3.814511e-07,0.009156,0.002316


In [60]:
y_test_All

,Label
0,1
1,0
2,0
3,0
4,0
...,...
88004,1
88005,0
88006,0
88007,0


In [61]:
# In train
print(Counter(list(y_train_withoutgraph['Label'])))
# In test
print(Counter(list(y_test_withoutgraph['Label'])))

Counter({0: 280919, 1: 71116})
Counter({0: 70231, 1: 17778})


# Some Binary Classifiers

In [28]:
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
XGB = xgboost.XGBClassifier(n_estimators=150, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='binary', n_estimators= 500, n_jobs=-1) # multiclass

list_of_CLFs_names = []
list_of_CLFs = [RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_withoutgraph, y_train_withoutgraph)
    pred = clf.score(X_test_withoutgraph, y_test_withoutgraph)
    pred_y = clf.predict(X_test_withoutgraph)
    recall, precision, fscore, _ = precision_recall_fscore_support(y_test_withoutgraph, pred_y, average='binary')
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f Recall: %0.5f Precision: %0.5f F1: %0.5f for the %s" % (pred, recall, precision, fscore, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

Acc: 0.99211 Recall: 0.98178 Precision: 0.97913 F1: 0.98046 for the RandomForestClassifier
Acc: 0.99155 Recall: 0.98064 Precision: 0.97744 F1: 0.97904 for the ExtraTreesClassifier
[07:40:12] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Acc: 0.99188 Recall: 0.98116 Precision: 0.97857 F1: 0.97986 for the XGBClassifier
Acc: 0.99219 Recall: 0.98266 Precision: 0.97863 F1: 0.98064 for the LGBMClassifier


In [30]:
GNB = GaussianNB()
# RBF_SVC = SVC(kernel='rbf')
REG = linear_model.LogisticRegression()

list_of_CLFs_names = []
list_of_CLFs = [GNB, REG]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_withoutgraph,y_train_withoutgraph)
    pred = clf.score(X_test_withoutgraph,y_test_withoutgraph)
    pred_y = clf.predict(X_test_withoutgraph)
    recall, precision, fscore, _ = precision_recall_fscore_support(y_test_withoutgraph, pred_y, average='binary')
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f Recall: %0.5f Precision: %0.5f F1: %0.5f for the %s" % (pred, recall, precision, fscore, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

Acc: 0.74816 Recall: 0.43911 Precision: 0.88953 F1: 0.58797 for the GaussianNB
Acc: 0.79966 Recall: 0.66009 Precision: 0.01693 F1: 0.03302 for the LogisticRegression


# With Additional Feature from Graph

## CliqueStar

In [31]:
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
XGB = xgboost.XGBClassifier(n_estimators=150, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='binary', n_estimators= 500, n_jobs=-1) # multiclass

list_of_CLFs_names = []
list_of_CLFs = [RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_CS, y_train_CS)
    pred = clf.score(X_test_CS, y_test_CS)
    pred_y = clf.predict(X_test_CS)
    recall, precision, fscore, _ = precision_recall_fscore_support(y_test_CS, pred_y, average='binary')
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f Recall: %0.5f Precision: %0.5f F1: %0.5f for the %s" % (pred, recall, precision, fscore, name))

    ranking.append(pred)

Acc: 0.99240 Recall: 0.98273 Precision: 0.97958 F1: 0.98115 for the RandomForestClassifier
Acc: 0.99171 Recall: 0.98082 Precision: 0.97806 F1: 0.97944 for the ExtraTreesClassifier
[07:42:27] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Acc: 0.99199 Recall: 0.98199 Precision: 0.97829 F1: 0.98013 for the XGBClassifier
Acc: 0.99243 Recall: 0.98366 Precision: 0.97879 F1: 0.98122 for the LGBMClassifier


In [32]:
GNB = GaussianNB()
# RBF_SVC = SVC(kernel='rbf')
REG = linear_model.LogisticRegression()

list_of_CLFs_names = []
list_of_CLFs = [GNB, REG]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_withgraph,y_train_withgraph)
    pred = clf.score(X_test_withgraph,y_test_withgraph)
    pred_y = clf.predict(X_test_withgraph)
    recall, precision, fscore, _ = precision_recall_fscore_support(y_test_withgraph, pred_y, average='binary')
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f Recall: %0.5f Precision: %0.5f F1: %0.5f for the %s" % (pred, recall, precision, fscore, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

Acc: 0.74816 Recall: 0.43911 Precision: 0.88953 F1: 0.58797 for the GaussianNB
Acc: 0.79966 Recall: 0.66009 Precision: 0.01693 F1: 0.03302 for the LogisticRegression


## HeavyVicinity

In [62]:
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
XGB = xgboost.XGBClassifier(n_estimators=150, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='binary', n_estimators= 500, n_jobs=-1) # multiclass

list_of_CLFs_names = []
list_of_CLFs = [RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_HV, y_train_HV)
    pred = clf.score(X_test_HV, y_test_HV)
    pred_y = clf.predict(X_test_HV)
    recall, precision, fscore, _ = precision_recall_fscore_support(y_test_HV, pred_y, average='binary')
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f Recall: %0.5f Precision: %0.5f F1: %0.5f for the %s" % (pred, recall, precision, fscore, name))

    ranking.append(pred)

Acc: 0.99224 Recall: 0.98245 Precision: 0.97908 F1: 0.98076 for the RandomForestClassifier
Acc: 0.99180 Recall: 0.98126 Precision: 0.97806 F1: 0.97966 for the ExtraTreesClassifier
[10:50:38] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Acc: 0.99230 Recall: 0.98251 Precision: 0.97930 F1: 0.98090 for the XGBClassifier
Acc: 0.99269 Recall: 0.98478 Precision: 0.97896 F1: 0.98186 for the LGBMClassifier


In [63]:
GNB = GaussianNB()
# RBF_SVC = SVC(kernel='rbf')
REG = linear_model.LogisticRegression()

list_of_CLFs_names = []
list_of_CLFs = [GNB, REG]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_HV,y_train_HV)
    pred = clf.score(X_test_HV,y_test_HV)
    pred_y = clf.predict(X_test_HV)
    recall, precision, fscore, _ = precision_recall_fscore_support(y_test_HV, pred_y, average='binary')
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f Recall: %0.5f Precision: %0.5f F1: %0.5f for the %s" % (pred, recall, precision, fscore, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

Acc: 0.75458 Recall: 0.44578 Precision: 0.88362 F1: 0.59260 for the GaussianNB
Acc: 0.79821 Recall: 0.52960 Precision: 0.00956 F1: 0.01879 for the LogisticRegression


## DominantPair

In [64]:
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
XGB = xgboost.XGBClassifier(n_estimators=150, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='binary', n_estimators= 500, n_jobs=-1) # multiclass

list_of_CLFs_names = []
list_of_CLFs = [RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_DP, y_train_DP)
    pred = clf.score(X_test_DP, y_test_DP)
    pred_y = clf.predict(X_test_DP)
    recall, precision, fscore, _ = precision_recall_fscore_support(y_test_DP, pred_y, average='binary')
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f Recall: %0.5f Precision: %0.5f F1: %0.5f for the %s" % (pred, recall, precision, fscore, name))

    ranking.append(pred)

Acc: 0.99224 Recall: 0.98228 Precision: 0.97924 F1: 0.98076 for the RandomForestClassifier
Acc: 0.99171 Recall: 0.98126 Precision: 0.97761 F1: 0.97943 for the ExtraTreesClassifier
[10:53:10] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Acc: 0.99234 Recall: 0.98207 Precision: 0.97998 F1: 0.98102 for the XGBClassifier
Acc: 0.99257 Recall: 0.98378 Precision: 0.97936 F1: 0.98157 for the LGBMClassifier


In [65]:
GNB = GaussianNB()
# RBF_SVC = SVC(kernel='rbf')
REG = linear_model.LogisticRegression()

list_of_CLFs_names = []
list_of_CLFs = [GNB, REG]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_DP,y_train_DP)
    pred = clf.score(X_test_DP,y_test_DP)
    pred_y = clf.predict(X_test_DP)
    recall, precision, fscore, _ = precision_recall_fscore_support(y_test_DP, pred_y, average='binary')
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f Recall: %0.5f Precision: %0.5f F1: %0.5f for the %s" % (pred, recall, precision, fscore, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

Acc: 0.74816 Recall: 0.43911 Precision: 0.88953 F1: 0.58797 for the GaussianNB
Acc: 0.79966 Recall: 0.66009 Precision: 0.01693 F1: 0.03302 for the LogisticRegression


## All features

In [66]:
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
XGB = xgboost.XGBClassifier(n_estimators=150, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='binary', n_estimators= 500, n_jobs=-1) # multiclass

list_of_CLFs_names = []
list_of_CLFs = [RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_All, y_train_All)
    pred = clf.score(X_test_All, y_test_All)
    pred_y = clf.predict(X_test_All)
    recall, precision, fscore, _ = precision_recall_fscore_support(y_test_All, pred_y, average='binary')
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f Recall: %0.5f Precision: %0.5f F1: %0.5f for the %s" % (pred, recall, precision, fscore, name))

    ranking.append(pred)

Acc: 0.99227 Recall: 0.98267 Precision: 0.97902 F1: 0.98084 for the RandomForestClassifier
Acc: 0.99168 Recall: 0.98066 Precision: 0.97812 F1: 0.97939 for the ExtraTreesClassifier
[10:56:44] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Acc: 0.99251 Recall: 0.98361 Precision: 0.97924 F1: 0.98142 for the XGBClassifier
Acc: 0.99271 Recall: 0.98423 Precision: 0.97958 F1: 0.98190 for the LGBMClassifier


In [67]:
GNB = GaussianNB()
# RBF_SVC = SVC(kernel='rbf')
REG = linear_model.LogisticRegression()

list_of_CLFs_names = []
list_of_CLFs = [GNB, REG]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_All,y_train_All)
    pred = clf.score(X_test_All,y_test_All)
    pred_y = clf.predict(X_test_All)
    recall, precision, fscore, _ = precision_recall_fscore_support(y_test_All, pred_y, average='binary')
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f Recall: %0.5f Precision: %0.5f F1: %0.5f for the %s" % (pred, recall, precision, fscore, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

Acc: 0.75458 Recall: 0.44578 Precision: 0.88362 F1: 0.59260 for the GaussianNB
Acc: 0.79821 Recall: 0.52960 Precision: 0.00956 F1: 0.01879 for the LogisticRegression


## only use graph feature data

In [69]:
X_train_onlygraph = train.loc[:, ['src_score_cs', 'src_score_hv', 'src_score_dp']]
y_train_onlygraph = train.loc[:,['Label']]
X_test_onlygraph = test.loc[:, ['src_score_cs', 'src_score_hv', 'src_score_dp']]
y_test_onlygraph = test.loc[:,['Label']]

In [70]:
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
XGB = xgboost.XGBClassifier(n_estimators=150, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='binary', n_estimators= 500, n_jobs=-1) # multiclass

list_of_CLFs_names = []
list_of_CLFs = [RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_onlygraph, y_train_onlygraph)
    pred = clf.score(X_test_onlygraph, y_test_onlygraph)
    pred_y = clf.predict(X_test_onlygraph)
    recall, precision, fscore, _ = precision_recall_fscore_support(y_test_onlygraph, pred_y, average='binary')
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f Recall: %0.5f Precision: %0.5f F1: %0.5f for the %s" % (pred, recall, precision, fscore, name))

    ranking.append(pred)

Acc: 0.96178 Recall: 0.99820 Precision: 0.81224 F1: 0.89567 for the RandomForestClassifier
Acc: 0.96178 Recall: 0.99820 Precision: 0.81224 F1: 0.89567 for the ExtraTreesClassifier
[11:00:27] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Acc: 0.96179 Recall: 0.99820 Precision: 0.81230 F1: 0.89570 for the XGBClassifier
Acc: 0.96178 Recall: 0.99820 Precision: 0.81224 F1: 0.89567 for the LGBMClassifier


In [71]:
GNB = GaussianNB()
# RBF_SVC = SVC(kernel='rbf')
REG = linear_model.LogisticRegression()

list_of_CLFs_names = []
list_of_CLFs = [GNB, REG]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_onlygraph,y_train_onlygraph)
    pred = clf.score(X_test_onlygraph,y_test_onlygraph)
    pred_y = clf.predict(X_test_onlygraph)
    recall, precision, fscore, _ = precision_recall_fscore_support(y_test_onlygraph, pred_y, average='binary')
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f Recall: %0.5f Precision: %0.5f F1: %0.5f for the %s" % (pred, recall, precision, fscore, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

Acc: 0.77205 Recall: 0.45682 Precision: 0.67955 F1: 0.54635 for the GaussianNB
Acc: 0.67943 Recall: 0.34839 Precision: 0.67443 F1: 0.45945 for the LogisticRegression
